In [1]:
!pip install modelscope==1.9.5
!pip install transformers==4.39.0
!pip install streamlit==1.24.0
!pip install sentencepiece==0.1.99
!pip install transformers_stream_generator==0.0.4
!pip install datasets==2.18.0
!pip install peft==0.10.0
!pip install openai==1.17.1
!pip install tqdm==4.64.1
!pip install transformers==4.39.3
!python -m pip install setuptools==69.5.1
!pip install vllm==0.4.0.post1
!pip install nest-asyncio
!pip install accelerate
!pip install tf-keras

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/17/d8/f808e32ed7fa86617b9ac7a37b7dcff894c839108c4871cc33ffc4e65b7d/datasets-2.13.0-py3-none-any.whl (485 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/a4/73/f620d76193954e16db3d5c53a07d956d7b9c800e570758d3bff91906d4a4/transformers-4.39.0-py3-none-any.whl (8.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [2]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
model_dir = snapshot_download('./qwen/Qwen2-7B-Instruct', cache_dir='./', revision='master')

2024-08-18 17:05:30,871 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-08-18 17:05:30,873 - modelscope - INFO - TensorFlow version 2.17.0 Found.
2024-08-18 17:05:30,873 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-08-18 17:05:30,983 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 3da902487d9387b55bb90991ee9123fc and a total number of 945 components indexed
Downloading: 100%|██████████| 663/663 [00:00<00:00, 758kB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 45.1kB/s]
Downloading: 100%|██████████| 243/243 [00:00<00:00, 269kB/s]
Downloading: 100%|██████████| 11.1k/11.1k [00:00<00:00, 9.11MB/s]
Downloading: 100%|██████████| 1.59M/1.59M [00:00<00:00, 8.52MB/s]
Downloading: 100%|█████████▉| 3.67G/3.67G [03:39<00:00, 18.0MB/s]
Downloading: 100%|█████████▉| 3.60G/3.60G [03:40<00:00, 17.5MB/s]
Downloading:  78%|███████▊  | 2.81G/3.60G [03:02<00:47, 17.8MB/s]2024-08-18 17:16:06,480 - modelscope

KeyboardInterrupt: 

In [3]:
model_dir = "./qwen/Qwen2-7B-Instruct"

## 这里运行完请重启notebook

In [4]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

2024-08-18 17:18:02.933013: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-18 17:18:02.947649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 17:18:02.964164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 17:18:02.968948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-18 17:18:02.981469: I tensorflow/core/platform/cpu_feature_guar

In [5]:
import pandas as pd
# 将JSON文件转换为CSV文件
train_data_size = 4000
ana = pd.read_json('data/ana.json')
qwen = pd.read_json('data/qwen_train_data_1697.json')
gsm8k2 = pd.read_json('data/gsm8k2_train_data_7473.json')
#gsm8k2.iloc[:num_records_needed]
temp_df = pd.concat([ana, qwen], ignore_index=True)
num_records_needed = train_data_size - len(temp_df)
gsm8k2 = gsm8k2.iloc[:num_records_needed]
combined_df = pd.concat([temp_df, gsm8k2], ignore_index=True)
shuffled_df = combined_df.sample()
ds = Dataset.from_pandas(combined_df.iloc[:int(train_data_size*0.8)])
dev_ds = Dataset.from_pandas(combined_df.iloc[int(train_data_size*0.8):])

In [6]:
tokenizer = AutoTokenizer.from_pretrained('./qwen/Qwen2-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

2024-08-18 17:18:08,824 - modelscope - WARNING - Download file from: 3858759680 to: 3864726407 failed, will retry
--- Logging error ---
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/site-packages/modelscope/hub/file_download.py", line 203, in download_part_with_retry
    with open(file_name, 'rb+') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/root/LLM_Logical_Challenge/temp/tmpd0i1wdei/tmp2bt831qv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/logging/__init__.py", line 1103, in emit
    stream.write(msg + self.terminator)
  File "/root/miniconda3/lib/python3.10/site-packages/ipykernel/iostream.py", line 662, in write
    self._schedule_flush()
  File "/root/miniconda3/lib/python3.10/site-packages/ipykernel/iostream.py", line 559, in _schedule_flush
    self.pub_thread.schedule(_schedule_in_thread)
  File "/root/miniconda3/lib/python3.10/sit

Qwen2Tokenizer(name_or_path='./qwen/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def process_func(example):
    MAX_LENGTH = 1800    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一个逻辑推理专家，擅长解决逻辑推理问题。<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id
dev_id = dev_ds.map(process_func, remove_columns=ds.column_names)
dev_id

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 800
})

In [9]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你是一个逻辑推理专家，擅长解决逻辑推理问题。<|im_end|>\n<|im_start|>user\n你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：\n\n### 题目:\n假设您需要构建一个二叉搜索树，其中每个节点或者是一个空的节点（称为"空节点"），或者是一个包含一个整数值和两个子树的节点（称为"数值节点"）。以下是构建这棵树的规则：\n\n1. 树中不存在重复的元素。\n2. 对于每个数值节点，其左子树的所有值都小于该节点的值，其右子树的所有值都大于该节点的值。\n3. 插入一个新值到一个"空节点"时，该"空节点"会被一个包含新值的新的数值节点取代。\n4. 插入一个已存在的数值将不会改变树。\n\n请基于以上规则，回答以下选择题：\n\n### 问题:\n选择题 1：\n给定一个空的二叉搜索树，插入下列数字: [5, 9, 2, 10, 11, 3]，下面哪个选项正确描述了结果树的结构？\nA. tree(5, tree(2, tree(3, nil, nil), nil), tree(9, tree(10, nil, nil), tree(11, nil, nil)))\nB. tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))\nC. tree(5, tree(3, tree(2, nil, nil), nil), tree(9, nil, tree(10, tree(11, nil, nil), nil)))\nD. tree(5, nil, tree(2, nil, tree(3, nil, nil)), tree(9, tree(11, nil, nil), tree(10, nil, nil)))\n<|im_end|>\n<|im_start|>assistant\n根据题目中的规则，我们依次插入数字 [5, 9, 2, 10, 11, 3] 来构建二叉搜索树。\n

In [10]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))


'根据题目描述的函数逻辑，我们来逐步分析输入列表 [3,7,5,9] 的处理过程：\n\n1. 列表非空，开始处理第一个元素3。\n2. 3是奇数，根据规则应被删除。\n3. 处理下一个元素7，同样为奇数，删除。\n4. 继续处理5，同样是奇数，删除。\n5. 最后处理9，依然是奇数，删除。\n\n整个过程中，由于输入列表中的所有元素都是奇数，根据规则它们都将被删除，不会有任何元素保留在结果列表中。\n\n因此，对于输入列表 [3,7,5,9]，函数返回的结果是空列表。\n\n答案是：D. []<|endoftext|>'

In [11]:
import torch

model = AutoModelForCausalLM.from_pretrained('./qwen/Qwen2-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [12]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [13]:
model.dtype

torch.bfloat16

In [14]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'q_proj', 'v_proj', 'k_proj', 'gate_proj', 'o_proj', 'down_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./qwen/Qwen2-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'up_proj', 'q_proj', 'v_proj', 'k_proj', 'gate_proj', 'o_proj', 'down_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [16]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.26434798934534914


In [17]:
# args = TrainingArguments(
#     output_dir="./output/Qwen2_instruct_lora",
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=4,
#     logging_steps=10,
#     num_train_epochs=1,
#     save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
#     learning_rate=1e-4,
#     save_on_each_node=True,
#     gradient_checkpointing=True
# )

args = TrainingArguments(
    output_dir="./output/Qwen2_instruct_lora/20240818",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=100,
    eval_steps=100,
    num_train_epochs=5,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    #bf16=True,
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    evaluation_strategy="steps"
    # save_strategy="steps"
)

In [19]:
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=tokenized_id,
#     data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
# )
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    eval_dataset=dev_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
100,0.644100,0.136285
200,0.551500,0.123918
300,0.579000,0.112671
400,0.555400,0.107636
500,0.436400,0.123495
600,0.458800,0.130354
700,0.451100,0.128521


/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.c

TrainOutput(global_step=700, training_loss=0.525200549534389, metrics={'train_runtime': 3320.0922, 'train_samples_per_second': 4.819, 'train_steps_per_second': 0.602, 'total_flos': 1.16871996654379e+17, 'train_loss': 0.525200549534389, 'epoch': 1.75})

In [23]:
torch.backends.cuda.enable_mem_efficient_sdp(False)

## 这里运行完请重启notebook

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '../qwen/Qwen2-7B-Instruct/'
lora_path = './output/Qwen2_instruct_lora/20240818/checkpoint-400' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.float16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = '''你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：\n\n### 题目:\n假设您需要构建一个二叉搜索树，其中每个节点或者是一个空的节点（称为"空节点"），或者是一个包含一个整数值和两个子树的节点（称为"数值节点"）。以下是构建这棵树的规则：\n\n1. 树中不存在重复的元素。\n2. 对于每个数值节点，其左子树的所有值都小于该节点的值，其右子树的所有值都大于该节点的值。\n3. 插入一个新值到一个"空节点"时，该"空节点"会被一个包含新值的新的数值节点取代。\n4. 插入一个已存在的数值将不会改变树。\n\n请基于以上规则，回答以下选择题：\n\n### 问题:\n选择题 1：\n给定一个空的二叉搜索树，插入下列数字: [5, 9, 2, 10, 11, 3]，下面哪个选项正确描述了结果树的结构？\nA. tree(5, tree(2, tree(3, nil, nil), nil), tree(9, tree(10, nil, nil), tree(11, nil, nil)))\nB. tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))\nC. tree(5, tree(3, tree(2, nil, nil), nil), tree(9, nil, tree(10, tree(11, nil, nil), nil)))\nD. tree(5, nil, tree(2, nil, tree(3, nil, nil)), tree(9, tree(11, nil, nil), tree(10, nil, nil)))'''
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一个逻辑推理专家，擅长解决逻辑推理问题。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')
#tensor_gpu = tensor.to('cuda:0')

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

为了确定正确的答案，我们按照给定的规则来构建二叉搜索树。

1. 首先，插入数字5，树变为：tree(5, nil, nil)
2. 接着插入数字2，因为2小于5，所以它成为5的左子节点：tree(5, tree(2, nil, nil), nil)
3. 然后插入数字3，同样地，3也小于5，所以它成为2的右子节点：tree(5, tree(2, nil, tree(3, nil, nil)), nil)
4. 插入数字9，9大于5，所以它成为5的右子节点：tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, nil))
5. 插入数字10，10大于5且大于9，所以它成为9的右子节点：tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, nil)))
6. 最后插入数字11，11大于5、9和10，所以它成为10的右子节点：tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))

因此，正确的答案是 B. tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))。

答案是：B


In [3]:
# 模型合并存储

new_model_directory = "./merged_model_an"
merged_model = model.merge_and_unload()
# 将权重保存为safetensors格式的权重, 且每个权重文件最大不超过2GB(2048MB)
merged_model.save_pretrained(new_model_directory, max_shard_size="2048MB", safe_serialization=True)


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
!cp ./qwen/Qwen2-7B-Instruct/tokenizer.json ./merged_model_an/

## 这里运行完请重启notebook